In [1]:
import yfinance as yf
import pandas as pd
import datetime as dt
import requests as req
from bs4 import BeautifulSoup
import utils
from statistics import mean
import cash_flow
import formulas


In [2]:
tickers = ["TCS", "GNFC"]

In [3]:
ticker_data = []
years = []

for i in range(10):
        years.append(utils.get_financial_year() + i)

forward_fcf_df = pd.DataFrame(index=years)

for idx, ticker in enumerate(tickers):
    free_cash_flows = cash_flow.scraped_fcf(ticker)
    growth_rates = cash_flow.calculate_growth_rate(free_cash_flows)

    ticker_data.append({ ticker: { "avg_growth_rate": mean(growth_rates), "last_fcf_value": free_cash_flows[len(free_cash_flows) - 1] } })

    forward_fcf_df[ticker] = cash_flow.generate_forward_fcf(10, ticker_data, idx, ticker)

    ticker_data[idx][ticker]["capm"] = formulas.CAPM(ticker)


print(ticker_data)




[{'TCS': {'avg_growth_rate': 26.247210724176576, 'last_fcf_value': 29120.0, 'capm': 5.91}}, {'GNFC': {'avg_growth_rate': 42.94427696184804, 'last_fcf_value': 541.15, 'capm': 5.91}}]


In [4]:
# PROJECT FUTURE FCF





Empty DataFrame
Columns: [2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]
Index: []


In [9]:
# Get Cashflows
apv_api_key = '2N8UEPVRC2AWTK6B'

# cashflows = req.get("https://www.alphavantage.co/query?function=OVERVIEW&symbol=TCS&apikey=2N8UEPVRC2AWTK6B")
# cashflows = cashflows.json()

stock = yf.Ticker("TCS.NS")

print(stock.info)
# annual_cf_df = pd.DataFrame(cashflows['annualReports']).sort_index(ascending=False)
# dates = annual_cf_df['fiscalDateEnding'].values.tolist()
# years = []

# for date in dates:
#     years.append(dt.datetime.strptime(date, "%Y-%m-%d").year)

{'zip': '400021', 'sector': 'Technology', 'fullTimeEmployees': 443676, 'longBusinessSummary': 'Tata Consultancy Services Limited provides information technology (IT) and IT enabled services worldwide. It operates through Banking, Financial Services and Insurance; Manufacturing; Retail and Consumer Business; Communication, Media and Technology; and Others segments. The company offers CHROMA, a cloud-based talent management solution; ignio, a cognitive automation software product; iON, an assessment platform; TAP, a procurement offering; TCS MasterCraft, a platform to automate and manage IT processes; and Quartz, a blockchain solution. It also provides customer intelligence and insight solutions to deliver retail, banking, and communications experiences; Intelligent Urban Exchange, an integrated software to accelerate smart city programs; OPTUMERA, a digital merchandising suite; TCS BaNCS, a financial platform; and Jile, an agile DevOps product. In addition, the company offers advanced d

In [57]:
fcf_df = pd.DataFrame()
fcf_df["Year"] = years
fcf_df["Operating Cashflow"] = annual_cf_df["operatingCashflow"].astype(int)
fcf_df["Cap Ex"] = annual_cf_df["capitalExpenditures"].astype(int)
fcf_df["FCF"] = fcf_df["Operating Cashflow"] - fcf_df["Cap Ex"]
growth_rate = [0]

for i in range(len(fcf_df["FCF"])):
    if i < len(fcf_df["FCF"]) - 1:
        pct_diff = (fcf_df["FCF"].iloc[i+1] - fcf_df["FCF"].iloc[i])/abs(fcf_df["FCF"].iloc[i])
        growth_rate.append(pct_diff * 100)

fcf_df["Growth Rate"]  = growth_rate
avg_growth = fcf_df["Growth Rate"].iloc[1:].mean()

avg_growth

-35.630076966509364